In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

In [2]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [3]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
model_bis = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

2.5725006639080936

In [6]:
from lipschitz_decomon_tools import get_local_maximum, echantillonner_boule_l2_simple

In [7]:
pt_choosen = 1
x = images[pt_choosen:pt_choosen+1].flatten().detach().cpu().numpy()
label = labels[pt_choosen:pt_choosen+1]
eps=0.5
nb_pts = 20

In [8]:
y_list = []
for i in range(nb_pts):
    y_list.append(echantillonner_boule_l2_simple(x, eps))

In [9]:
# x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)

In [10]:
from radius_evaluation_tools import single_compute_relaxation_radius

In [ ]:
eps_working = single_compute_relaxation_radius(pt_choosen, images, labels, model, y_list, 3)